### Queries na base de dados Mongo


In [ ]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [ ]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [ ]:
user_posgres = cfg["postgres"]["user"]
passwd_posgres = cfg["postgres"]["passwd"]
host_posgres = cfg["postgres"]["host"]
dbmongo_posgres = cfg["postgres"]["db"]

In [ ]:
host_posgres

In [ ]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [ ]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [ ]:
col_processos.count_documents({})

In [ ]:
#df = pd.DataFrame.from_records(processos,index=['nro_processo','link_processo','nome_parte'])
dados = pd.DataFrame(list(col_processos.find({})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

In [ ]:
# Processos por empresa
df_processos.groupby('cod_empresa')['nro_processo'].count()

In [ ]:
df_processos.columns

In [ ]:
from sqlalchemy import create_engine
import psycopg2

In [ ]:
conn_string_postgres = ''
  
db = create_engine(conn_string_postgres)
conn = db.connect()

In [ ]:
df_processos.to_sql('dw_processos', con=conn, if_exists='replace',index=False)

# Decisões

In [ ]:
# Carrega base de decisões total
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])

In [ ]:
df_decisoes.shape

In [ ]:
df_decisoes.to_sql('dw_decisoes', con=conn, if_exists='replace',index=False)

In [ ]:
df_final = pd.merge(df_processos,df_decisoes,on="nro_processo")

In [ ]:
df_final.columns

In [ ]:
df_final = df_final[['nro_processo', 'link_processo', 'nome_parte', 'classe_processo','assunto_processo', 
'data_local_distribuicao', 'cod_empresa','tipo_participacao', 'classe', 'assunto', 'magistrado', 'comarca',
'foro', 'vara', 'data_disp', 'instancia', 'resultado']]

In [ ]:
df_final['tribunal'] = 'TJSP'
df_final['justica'] = 'Justiça Estadual'

In [ ]:
df_final.to_excel('../../dados/analiticos/base_processos.xlsx',sheet_name="DADOS",index=False)

In [ ]:
col_decisoes.count_documents({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})

In [ ]:
col_decisoes.update_many({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}},{'$set':{'resultado':'Parcialmente Procedente'}})

In [ ]:
decisao = col_decisoes.find_one({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})
decisao['decisao']

In [ ]:
# Decisões por Magistrado
df_qtdedecisoes = pd.DataFrame({"decisoes" : df_decisoes.groupby('magistrado').size()}).reset_index()
df_qtdedecisoes.sort_values(by="decisoes",ascending=False)